In [2]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import torch 
from model1.spanemo.model import SpanEmo
from model1.spanemo.data_loader import DataClass
from fastprogress.fastprogress import format_time, master_bar, progress_bar
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import datetime
import numpy as np
import json
import math

In [3]:
df = pd.read_csv("cornell_movie/predictions.csv")

In [4]:
df.drop()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,utterance text,speaker,conversation_id,0,1,2,...,18,19,20,21,22,23,24,25,26,27
0,0,0,0,0,They do not!,u0,L1044,-8.052365,-8.175057,-6.226021,...,-8.333762,-8.336207,-7.862222,-8.547613,-6.139152,-8.437870,-8.259851,-7.689548,-8.167355,2.519341
1,1,1,1,1,They do to!,u2,L1044,-7.363643,-7.539609,-6.305096,...,-7.726948,-8.030384,-6.329423,-8.293463,-5.284000,-7.944016,-8.296870,-7.327426,-7.314297,3.753215
2,2,2,2,2,I hope so.,u0,L984,-7.674916,-8.189976,-8.220063,...,-8.028004,-8.382309,3.741410,-8.533669,-7.790706,-8.272712,-8.568789,-8.038083,-8.474198,-5.249081
3,3,3,3,3,She okay?,u2,L984,-7.802964,-8.037459,-7.943669,...,-8.027291,-7.490631,-7.594229,-8.463768,-7.411293,-6.965907,-8.347673,-7.013407,-7.727482,0.628213
4,4,4,4,4,Let's go.,u0,L924,-7.713368,-7.858588,-6.787925,...,-8.113943,-8.096422,-5.721414,-8.452881,-5.515314,-8.016721,-8.270496,-7.152738,-7.782495,3.867799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164680,164680,164680,164680,164901,Sell maps?,u4749,L153138,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164681,164681,164681,164681,164902,Only two bucks. Shave as well...,u4739,L153132,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164682,164682,164682,164682,164903,Thanks. Just some pills.,u4749,L153132,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164683,164683,164683,164683,164904,Looks like you need a haircut to me.,u4739,L153132,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
seed = 12345678

if str(device) == 'cuda:0':
    print("Currently using GPU: {}".format(device))
    np.random.seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    print("WARNING: USING CPU")

Currently using GPU: cuda:0


In [3]:
paths_dict = {
    "cornell": "cornell_movie/predictions.csv",
    "iemocap": "iemocap/predictions.csv", #do ur stuff here
    "another": "another_df/predictions.csv", # do ur stuff here
}

df_dict = {
    "cornell": pd.read_csv(paths_dict["cornell"]),
    "iemocap": None,
    "another": None,
}

text_dict = {
    "cornell": np.array(df_dict["cornell"]["utterance text"]),
    "iemocap": np.array(0) # whatever
}

dataset_name = "cornell"
data_path = paths_dict[dataset_name]
pred_df = df_dict[dataset_name]
text_arr = text_dict[dataset_name]

In [4]:
hyperparams = {
    'backbone':'bert-base-uncased',
    'max_length': 128, #maximum context length for the model
    'batch_size': 128,
}

In [5]:
model_path = "../model1/final_model.pt"
model = SpanEmo()
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SpanEmo(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [7]:
x = 32768
i = 0
dataset = DataClass(hyperparams, text_arr[i*x:i*x+x], pred_mode=True)
data_loader = DataLoader(dataset,
                        batch_size=hyperparams['batch_size'],
                        shuffle=False
                        )

Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...
Reading twitter_2018 - 1grams ...


PreProcessing dataset ...: 100%|██████████| 32768/32768 [00:27<00:00, 1175.01it/s]


In [8]:
index = 0

In [9]:
with torch.no_grad():
    current_index = 0
    for step, batch in enumerate(progress_bar(data_loader, parent=None, leave=False)):
        if step < index:
            current_index += hyperparams['batch_size']
            continue

        num_rows, y_pred, logits = model.predict(batch, device)
        
        pred_df.iloc[i*x + current_index: i*x + current_index + num_rows, pred_df.columns.get_loc("0"):] = logits.cpu().numpy()
        if step % 10 == 0 and step > 0: pred_df.to_csv(data_path)
        
        current_index += num_rows

pred_df.to_csv(data_path)

In [10]:
def loop(start=0, x=32768):
    model_path = "../model1/final_model.pt"
    model = SpanEmo()
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    
    mb = master_bar(range(start, math.ceil(len(pred_df)/x)))
    for i in mb:
        dataset = DataClass(hyperparams, 
                            text_arr[i*x : min(i*x+x, len(pred_df))], 
                            pred_mode=True, 
                            pbar=lambda x, **kw: progress_bar(x, parent=mb, leave=False))
        data_loader = DataLoader(dataset,
                                batch_size=hyperparams['batch_size'],
                                shuffle=False
                                )
        
        with torch.no_grad():
            current_index = 0
            for step, batch in enumerate(progress_bar(data_loader, parent=mb, leave=False)):
                num_rows, y_pred, logits = model.predict(batch, device)
                
                pred_df.iloc[i*x + current_index: i*x + current_index + num_rows, pred_df.columns.get_loc("0"):] = logits.cpu().numpy()
                if step % 16 == 0 and step > 0: pred_df.to_csv(data_path)
                
                current_index += num_rows

        pred_df.to_csv(data_path)

In [11]:
loop(start=0)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...
Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...
Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...
Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...
